# Notebook N1 : generate rhyming poems

This notebooks allows to generate rhyming poems. First, given a form, it generates a non-rhyming poem correposponding to stage 1 of the CRPO system, and then, given a rhyming pattern it re-generates last words in order for it to rhyme, corresponding to stage 4.

- Make sure to define the variables you want in "0. Arguments".
- Sections 1-4 are just funtion definitions.
- In Section 5 you call the functions: load and build the model, generate non-rhyming poem, generate rhyming poem out of previous.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 38.7 MB/s 
     |████████████████████████████████| 182 kB 60.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.3 MB/s 


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from calendar import c
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from string import punctuation
from transformers import pipeline
import numpy as np
import re
from datetime import datetime
import random
import os

import torch
import nltk
from nltk.tokenize import word_tokenize
!pip3 install pickle5
import pickle5 as pickle
import difflib
import time

import sys
sys.path.insert(0, '..')
#from content.drive.MyDrive.crpo.crpo-notebooks.correct_with_dictionary import postprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 37.1 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236293 sha256=7ad3378bff9f0ebfe6a6381c0f63a16ae0bd62b02783a13c11b5fbdffdd8d78a
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [ ]:
import difflib
import numpy as np
from nltk import word_tokenize
import re
import random


def postprocessing (text, punctuation):
    """
    Does a bit of aesthetic postprocessing on the text. Each part is separate and ad hoc quick patch atm, so
    things could be improved, added, or deleted - there's lots of redundancies (should be cleaned up).
    """

    # Don't start with punctuation or ' '.
    while text[0] in punctuation:
        text = text[1:]

    # Punctuation immediately follows alphabetical characters (no ' ' in between).
    idxs_punctuation = []
    for i in range(len(text) - 1):
        if text[i] in punctuation[:-1]:
            if text[i-1] == ' ':
                idxs_punctuation += [i - 1]
    text = [text[i] for i in range(len(text)) if i not in idxs_punctuation]

    # Don't have two punctuation signs next to each other.
    idxs_punctuation = []
    for i in range(len(text) - 1):
        if text[i] in punctuation:
            if text[i+1] in punctuation:
                idxs_punctuation += [i + 1]
    text = [text[i] for i in range(len(text)) if i not in idxs_punctuation]

    # Add ' ' after punctuation.
    new_text = []
    for i in range(len(text) - 1):
        if text[i] not in punctuation:
            new_text += [text[i]]
        elif text[i] == ' ':
            new_text += [text[i]]
        elif text[i] in punctuation[:-1]:
            new_text += [text[i], ' ']
    text = new_text

    # No ' ' after ' ', '↵' into '\n', no verse start with ' ', write '...' correctly.
    text_joined = ''.join(text)
    text_joined = re.sub(r" +", ' ', text_joined)
    text_joined = re.sub(r"↵", '\n', text_joined)
    text_joined = re.sub(r'\n ','\n', text_joined)
    text_joined = re.sub('\.\.+', '...', text_joined)

    # Correct uppercase and lowercase #The new gpt-2 model doesn't generally makes casing mistakes 
    #text = [i for i in text_joined]
    #text[0] = text[0].upper()
    #for i in range(len(text)):
        #if text[i-1] in ['.', '!', '?']:
            #if text[i] == ' ' and i+1 < len(text):
                #text[i+1] = text[i+1].upper()
            #else:
                #text[i] = text[i].upper()
        #elif text[i-1] in [',', ':', ';']:
            #text[i] = text[i].lower()

    # Delete some t percent of ending soft punctuation to make it look a bit more natural.
    t = 0.25
    idxs_to_delete = []
    for i in range(len(text)):
        if text[i] == '\n':
            if text[i-1] != '\n':
                end = text[i-2]
                if end in [',', ':', ';']:
                    if random.random() < t:
                        idxs_to_delete += [i-2]
    text = [text[i] for i in range(len(text)) if i not in idxs_to_delete]

    # Add ' ' before certain punctuation signs.  
    #new_text = []
    #for i in range(len(text) - 1):
        #if text[i+1] not in [':', ';', '!', '?']:
            #new_text += [text[i]]
        #else:
            #new_text +=[text[i], ' ']
    #new_text += text[-1]
    #text = new_text

    # No ' ' before '\n' and end strophes with punctuation
    re_sub = re.sub(r' \n', '\n', ''.join(text))
    text = [i for i in re_sub]
    new_text = []
    for i in range(len(text) - 2):
        if text[i+1] == '\n' and text[i+2] == '\n':
            if text[i] in ['!', '?', '.']: #While I don't like the idea of random hard punctuation, I think it's better than nothing.
                new_text += [text[i]]
            elif text[i] in [';', ',', ':']: 
                new_text += [text[i]] #But not better than soft punctuation.
            else:
                new_text += [text[i], random.choice(['!', '?', '.'])]
        else:
            new_text += [text[i]]
    new_text += text[-2:]
    text = new_text

    # End with hard punctuation.
    if  text[-2] in punctuation and text[-2] not in ['!', '?', '.']:
        text[-2] = random.choice(['!', '?', '.'])
        text[-1] = ""
    elif text[-2] in ['!', '?', '.']:
        text[-1] = ""
    else:
        text[-1] = random.choice(['!', '?', '.'])

    return ''.join(text)



## 0. Arguments

In [ ]:
# -1 for CPU, 0 to n for CUDA devices (like a NIVDIA GPU)
DEVICE = 0

In [ ]:
OUTPUT_DIRECTORY_FILE = '../logs/'

In [ ]:
# Define form
# approximations for sonnet
s = 4
sonnet_l = 10 #should be 12, but let's speed up generation
HAIKU = [[7*s, 9*s, 7*s], [8*s, 11*s, 8*s]]
SONNET = [[sonnet_l*s]*4, [sonnet_l*s]*4, [sonnet_l*s]*3, [sonnet_l*s]*3]
QUATRAIN = [[10*s]*4]

In [ ]:
# RIME_PICKLE_FILE = '../utils/data/cons_rime_eng.pickle'
RIME_PICKLE_FILE = "/content/drive/MyDrive/crpo/crpo-notebooks/rhyming_dictionaries.pickle" # updated phonetic dictionary

In [ ]:
RHYMING_SCHEME = "AABB"

## 1. Load & Build Model Functions

In [ ]:
def load_models (models, path):
    """
    Loads and builds the trained language models, their vocab and indices, given path and models' names.
    """
    built_models = []

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    
    model_path = os.path.join(path, models[0])
    model = pipeline('text-generation', model=model_path, tokenizer='gpt2', device = DEVICE, pad_token_id=tokenizer.eos_token_id)
    built_models.append(model)

    model = GPT2LMHeadModel.from_pretrained(model_path)
    built_models.append(model)

    built_models.append(tokenizer)

    return [built_models]

## 2. Syllabification Functions

In [ ]:
def syllable_count (word) : # Found this algorythm on this website : https://eayd.in/?p=232. Counts the syllables of a word following some rules

    word = word.lower()
    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

def syllable_count_sentence(sent): # Counts the syllables of a sentence
    count = 0
    sent = sent.replace("\n", "")
    sent_tokens = sent.split(" ")
    for word in sent_tokens:
            word = word.strip(punctuation)
            if word != "":
                count += syllable_count(word)
    return count

def flatten(L):
    """
    Flattens a list.
    """

    return [L] if not isinstance(L, list) else [x for X in L for x in flatten(X)]

## 3. Form Generation Functions

In [ ]:
def generate(form,
            models,
            outfile_directory,
            temperature= 1.0,
            print_into_file=False, 
            d=False,
            user_input=""):
   
    text = "<start>" + user_input
    number_of_strophes = len(form)
    punctuation = ['.', ',', ':', ';', '!', '?', ' ']
    verse_nb = 0
    first_verse = True
    input_length = syllable_count(user_input)

    if(d):
        special_gen_nb = 0

    # Start generation of poem
    for s in range(number_of_strophes): #Strophes

        number_of_verses = form[s]
    
        for v in range(len(number_of_verses)): #Verses
            verse_length = number_of_verses[v]
            verse_nb_syllables = round(verse_length / 5)
            verse_nb_tokens = round(verse_nb_syllables + ((verse_nb_syllables) / 2)) #Number of tokens in verse + one special token every two syllables
            verse = ""
            nb_generations = 0
            special_gen = False
            forbidden_tokens_special_gen = [[29],[9688],[27],[198],[220],[532],[60],[62],[26],[4210],[32],[960],[7],[27920], [685], [2361], [1391], [1782], [4808], [1279]] # tokens for \n<start> = 198, 27, 9688, 29,

            if not first_verse:
                input_length = 0

            max_lenght_verse = verse_nb_tokens + 2 + 4 - input_length
            min_lenght_verse = verse_nb_tokens - 2 + 4 + round(len(text) / 4)- input_length

            # Generate verse from the existing text
            generated = ""
            # Check if at least a full verse has been generated and if the verse has the correct number of syllables
            while generated.count('<start>') < verse_nb + 2 or syllable_count_sentence(generated.split("<start>")[verse_nb + 1]) != verse_nb_syllables:
                
                if nb_generations < 10: # Generate another verse up to 10 times
                    generated = models[0](text, temperature = temperature, max_new_tokens = max_lenght_verse, min_length = min_lenght_verse)[0]['generated_text']
                    nb_generations += 1
                
                else: # If the verse has not been generated after 10 tries, generate a new verse token by token
                    if(d):
                        print("special gen")
                        special_gen_nb += 1

                    special_gen = True 
                    s = input_length
                    if(s < verse_nb_syllables) : # This is necessary because with the user input the verse could already have the correct number of syllables
                        #This first generation take the original text as input
                        generated = models[0](text, temperature = temperature, max_new_tokens = 1, bad_words_ids = forbidden_tokens_special_gen )[0]['generated_text']
                    else:
                        generated = text
                    while s < verse_nb_syllables:
                        #the remaining generations take the original text plus the new tokens as an input
                        generated = models[0](generated, temperature = temperature, max_new_tokens = 1, bad_words_ids = forbidden_tokens_special_gen)[0]['generated_text']
                        s = syllable_count_sentence(generated.split("<start>")[verse_nb + 1])
                    generated += "\n<start>"
                    
                    
                        
            verse = generated.split("<start>")[verse_nb + 1] #With this we separate the new verse form the rest of the text
            
            if special_gen:
                verse = verse.replace("-","")
                print('special gen')
            if first_verse:
                verse = verse.replace(user_input,"")
                first_verse = False

            if (d):
                print("Nb gen :" , nb_generations)
                print(verse)
                

            if v == (len(number_of_verses) - 1):
                text = text + verse
            else:
                 text = text + verse  + "<start>"
            verse_nb += 1

        text = text + "\n<start>"
    
    text = text.replace("<start>", "")
    if(d):
        print("Temperature:" , temperature)
        print("Generated poem : \n" , text)
        print("Nb of special gen:" , special_gen_nb)
        print('\n')


    # Postprocessing.
    text = postprocessing(text, punctuation)


    # Print poem into file
    if print_into_file:
        print('ahbon')
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S_GEN") #Windows compatible
        with open('lesgo' + '.txt', 'a') as f:
            print('lesgo')
            f.write(text)

    if d:
        print("Final poem: \n" , text)
        print('\n')

    return text

## 4. Rhyme Generation Functions

In [ ]:
def get_rhyming_dictionary (path=RIME_PICKLE_FILE):
    """Dictionaries of {"word": [perfect rhyme], assonant rhyme},
    and the inversions, to have 3 total dictionaries.
    """
    pickle_in = open(path,"rb")
    word2rhymes, cons_rhyme2words, assonant_rhyme2words = pickle.load(pickle_in)

    return word2rhymes, cons_rhyme2words, assonant_rhyme2words


def tokenize_poem (text):
    """
    Tokenize poem into strophes and verses.
    """
    structured = []
    strophes = text.split('\n\n')
    for s in strophes:
        strophe = []
        verses = s.split('\n')
        for v in verses:
            strophe += [v]
        structured += [strophe]
    return structured

def process_verse (verse):
    """
    Given a verse, returns its final punctuation (if any), last word, and remainder
    of verse without either of the two.
    """

    punctuation = ['.', ',', ',', ':', ';', '!', '?', ' ', '-']

    tok_verse = word_tokenize(verse, language='english') #changed to english -Teo
    # Remember punctuation
    if tok_verse[-1] in punctuation:
        original_punctuation = [tok_verse[-1]]
    else:
        original_punctuation = ''

    # Get last word
    if tok_verse[-1] in punctuation:
        if len(tok_verse) > 1:
            last_word = tok_verse [-2]
        else:
            last_word = ''
    else:
        last_word = tok_verse [-1]
    if "'" in last_word:
        last_word = last_word.split("'")[1]

    # Get new verse
    try:
        while tok_verse[-1] in punctuation:
            tok_verse = tok_verse[:-1]
    except IndexError:
        pass

    new_verse = ' '.join(tok_verse[:-1])
    new_verse = re.sub (r" â€™ ",r'â€™', new_verse)
    new_verse = re.sub (r" \.", '.', new_verse)

    return original_punctuation, last_word, new_verse

def get_candidates_rime (word_to_replace, word, dictios):
    """
    Given a word and an dictionary, checks if word is in dictionary. If not, checks for closest.
    If yes/with closest word: returns its consonant and assonant rhymes.
    If there are consonant rhymes candidates, returns them, otherwise, assonant.
    If none, returns an empty list.
    """
    
    word2rhymes, cons_rhyme2words, assonant_rhyme2words = dictios
    
    word = word.lower()
    if word not in word2rhymes:
        # find similar words
        try:
            word = difflib.get_close_matches(word, list(word2rhymes.keys()))[0]
        except:
            word = difflib.get_close_matches(word, list(word2rhymes.keys()), cutoff=0.25)[0]
        
    cons_rhyme = word2rhymes[word][0]
    cons_rhyme_candidates = [w for w in cons_rhyme2words[cons_rhyme] if w != word]
    # check if there are words that have consonant rhyme
    if len(cons_rhyme_candidates) != 0:
        rhyme = cons_rhyme
        candidates = cons_rhyme_candidates
    # if not, check if there are words that have assonant rhyme
    else:
        assonant_rhyme = word2rhymes[word][1]
        assonant_rhyme_candidates = [w for w in assonant_rhyme2words[assonant_rhyme] if w != word]
        if len(assonant_rhyme_candidates) != 0:
            rhyme = assonant_rhyme
            candidates = assonant_rhyme_candidates
        # if not, return original word
        else:
            rhyme = ""
            candidates = []
    
    return rhyme, candidates


def seq_prob (verse_idx, strophe, old_verse, candidate, models):
    """
    Calculates probability of a sequence of old_verse + ending (candidate) word.
    """
    
    #candidate = candidate.to_numpy()
    candidate_in_context = []
    for i in range(len(strophe)):
        candidate_in_context.append("<start>" + strophe[i])
    candidate_in_context[verse_idx] = "<start>" + old_verse + " " + candidate

    candidate_in_context = "\n".join(candidate_in_context)

    
    tokenize_input = models[2].tokenize(candidate_in_context)
    tensor_input = torch.tensor([models[2].convert_tokens_to_ids(tokenize_input)])
    begin = time.time() #START ===============================================
    loss = models[1](tensor_input, labels=tensor_input)
    end = time.time()
    print(f"Total runtime of the model program is {end - begin}") #END ===========
    return -loss[0].item()


import asyncio

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)
    return wrapped

@background
def seq_prob (verse_idx, strophe, old_verse, candidate, models):
  candidate_in_context = []
  for i in range(len(strophe)):
    candidate_in_context.append("<start>" + strophe[i])
  candidate_in_context[verse_idx] = "<start>" + old_verse + " " + candidate
  candidate_in_context = "\n".join(candidate_in_context)
  tokenize_input = models[2].tokenize(candidate_in_context)
  tensor_input = torch.tensor([models[2].convert_tokens_to_ids(tokenize_input)])
  loss = models[1](tensor_input, labels=tensor_input)
  return -loss[0].item()


def apply_rhyming_scheme (poem,
                          models,
                          rhyming_scheme,
                          outfile_directory=OUTPUT_DIRECTORY_FILE,
                          print_into_file=False,
                          d=False,
                          n_candidates=30):
    """
    Àlex: documentation is not up to date.
    Gets a poem raw (as the first stage generates it), and according to the rhyming scheme
    (e.g., ABBA ABBA CCD EDE, etc. [which is now missing]), returns it rhyming.
    n_candidates = for how many candidate words are we going to calculate their probability.
    rime_syllables = number of syllables that should rhyme.
    print_into_file is used to DEBUG only (save to disk before and after rhyme)
    Morever, 'd' allows printing of detailed debug info in TERMINAL
    """
    
    punctuation = ['.', ',', ',', ':', ';', '!', '?', ' ', '-']
    tok_poem = tokenize_poem(poem) # segment poem into stanzas and verses.

    new_poem = []
    new_poem_markup = []
    used_candidates = []
    strophes_scheme = rhyming_scheme.split(' ')
    
    rhyming_dictios = get_rhyming_dictionary()

    all_rhymes = dict([(c, '') for c in set(rhyming_scheme) if not c == ' '])

    for strophe_idx in range(len(tok_poem)):
        strophe = tok_poem[strophe_idx]
        new_strophe = []
        new_strophe_markup =[]

        begin = time.time() #START ===============================================

        for verse_idx in range(len(strophe)):
            full_old_verse = strophe[verse_idx]
            all_probs = []
            punct, word, old_verse = process_verse(full_old_verse)
            verse_scheme = strophes_scheme[strophe_idx][verse_idx]
            if d: 
                print('\tStanza: {} Verse: {} Rhyme: {}'.format(strophe_idx+1, verse_idx+1, verse_scheme))
                print(all_rhymes[verse_scheme])

            if all_rhymes[verse_scheme] == '':  # If it is a new rhyme leave it as is
                all_rhymes[verse_scheme] = word
                final_word = word
                final_word_markup = word
                rime = 'nochange'
            else: # If it is not a new rhyme
                rime, candidates = get_candidates_rime (word, all_rhymes[verse_scheme], rhyming_dictios)
                
                if d: print("#1a", candidates[:100])
                candidates = [i for i in candidates if i not in used_candidates and len(i) > 1]

                if d: print("#1b", candidates[:100])

                if len(candidates) == 0:
                    candidates = [word]                       

                if len(candidates) < n_candidates:
                    n_candidates = len(candidates)

                random.shuffle(candidates)
                random_candidates = random.sample(candidates, n_candidates)



                begin = time.time() #START ===============================================

                all_probs = []
                for cand in random_candidates:
                    all_probs.append(seq_prob(verse_idx, strophe, old_verse, cand, models).result())



                end = time.time()
                print(f"Total runtime of the for parallel program is {end - begin}") #END ===========
                
                begin = time.time() #START ===============================================
                if d: 
                    print('\t\tALL CANDIDATES:', len(candidates), '\n')
                    if len(candidates) < 100:
                        print('\t\t', word, rime, candidates, '\n')
                
                #all_probs = rand_cand.apply(lambda cand: seq_prob(verse_idx, strophe, old_verse, cand, models))
                
                
                #for candidate in random_candidates: #pndidatesararalelize random ca
                    #prob = seq_prob (verse_idx, strophe, old_verse, candidate, models) 
                    # vu l'appel de apply_rhyming_scheme
                    # on n'aurait ici que la pipeline avec les modÃ¨les
                    #all_probs += [prob]

                end = time.time()
                print(f"Total runtime of the for candidates program is {end - begin}") #END ===========

                begin = time.time() #START ===============================================
                print(all_probs)
                highest_to_lowest = np.argsort(all_probs)[::-1] # why sort all if we only need max?
                end = time.time()
                print(f"Total runtime of the argsort program is {end - begin}") #END ===========

                best = highest_to_lowest[0]
                best_prob = all_probs[best]
                final_word = random_candidates[best]
                final_word_markup = "[u]" + final_word + "[/u]"

            if len(punct) != 0:
                best_riming_verse = old_verse + ' ' + ''.join(final_word) + punct[0]
                best_riming_verse_markup = old_verse + ' ' + ''.join(final_word_markup) + punct[0]
            else:
                best_riming_verse = old_verse + ' ' + ''.join(final_word)
                best_riming_verse_markup = old_verse + ' ' + ''.join(final_word_markup)

            used_candidates += [final_word]

            best_riming_verse += '\n'
            best_riming_verse_markup += '\n'

            new_strophe += [best_riming_verse]
            new_strophe_markup += [best_riming_verse_markup]

        end = time.time()
        print(f"Total runtime of the chargin program is {end - begin}") #END ===========

        new_strophe += ['\n']
        new_strophe_markup += ['\n']

        new_poem += [new_strophe]
        new_poem_markup += [new_strophe_markup]

    raw = ''.join((flatten(new_poem)))
    new_poem = postprocessing(raw, punctuation)

    raw_markup = ''.join((flatten(new_poem_markup)))
    new_poem_markup = postprocessing(raw_markup, punctuation)

    if d: 
        print('NEW POEM\n\n' + new_poem + '\n' + '~'*50 + '\n' + 'OLD POEM\n\n' + poem)
    if print_into_file:
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S_RHYME") #Windows compatible
        with open('10Mo' + '.txt', 'a') as f:
            #f.write('NEW POEM\n\n' + new_poem + '\n' + '~'*50 + '\n' + 'OLD POEM\n\n' + poem)
            f.write(new_poem )
        with open('/content/drive/MyDrive/crpo/' + '10Mo' + '.txt', 'a') as f:
            #f.write('NEW POEM\n\n' + new_poem + '\n' + '~'*50 + '\n' + 'OLD POEM\n\n' + poem)
            f.write(new_poem )

    return (new_poem_markup, new_poem)

## 5. Function Calls

In [ ]:
begin = time.time()
MODEL = load_models (['gpt2-poetry-model-with-start-token_100_epochs'], "/content/drive/Shareddrives/ML4Science/")[0] #CHANGE HERE FOR NEW MODEL
end = time.time()
print(f"Total runtime of the loading is {end - begin}")


for i in range(100):
    non_rhyming_poem = generate(QUATRAIN,
                                MODEL,
                                OUTPUT_DIRECTORY_FILE,
                                temperature= 1.0,
                                print_into_file=False,
                                d=False,
                                user_input='')
    #print('===================================================')
    print('Here goes the poem generated : ')
    #print('===================================================')

    #print(non_rhyming_poem) #now it should rhyme
    with open('outputs.txt', 'a', encoding='utf-8') as w:
        w.write(non_rhyming_poem)
    
    
    #begin = time.time()
    '''
    rhyming_poem_markup, rhyming_poem = apply_rhyming_scheme (non_rhyming_poem,
                                                              MODEL,
                                                              RHYMING_SCHEME,
                                                              outfile_directory=OUTPUT_DIRECTORY_FILE,
                                                              print_into_file=True,
                                                              d=False,
                                                              n_candidates=30)
    '''
    #print(rhyming_poem)
    #end = time.time()
    #print(f"Total runtime of the program is {end - begin}")